In [1]:
from ase import Atoms
from ase.io import read
import xcquinox as xce
import torch, jax, optax
import numpy as np
import equinox as eqx
import jax.numpy as jnp
import pyscfad as psa
import os, sys
from pyscf import dft, scf, gto, df
from pyscf.pbc import scf as scfp
from pyscf.pbc import gto as gtop
from pyscf.pbc import dft as dftp
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'

from mp_api.client import MPRester
from mldftdat.density import get_exchange_descriptors2
from mldftdat.analyzers import RKSAnalyzer

from ase.build import bulk

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '
CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12030, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
a1 = bulk('Si', a=3.867114, b=3.867114, c=3.867114, alpha=60)

In [3]:
a1.cell

Cell([[0.0, 1.933557, 1.933557], [1.933557, 0.0, 1.933557], [1.933557, 1.933557, 0.0]])

In [4]:
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []

cell = gtop.Cell()
a = 5.43
cell.atom = [['Si', [0,0,0]],
              ['Si', [a/4,a/4,a/4]]]
cell.a = jnp.asarray([[0, a/2, a/2],
                     [a/2, 0, a/2],
                     [a/2, a/2, 0]])
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build()
kpts = cell.make_kpts([2,2,2])
mf = dftp.RKS(cell, xc='pbe0')
mf2 = dftp.KRKS(cell, xc='pbe0', kpts=kpts)
e = mf.kernel()
# e2 = mf2.kernel()

<class 'pyscf.pbc.dft.rks.RKS'> does not have attributes  nlc nlcgrids
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/pbc/gto/cell.py:243: UserWarning: Function cell.dumps drops attribute a because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -7.21836790602506


In [5]:
mf

RKS-KohnShamDFT object of <class 'pyscf.pbc.dft.rks.RKS'>

In [6]:
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []

mfs.append(mf)
dm = mf.make_rdm1()
dmj = jnp.array(dm)
dmj.flags = dm.flags
ao_eval = jnp.array(mf._numint.eval_ao(mf.mol, mf.grids.coords, deriv=2))
energies.append(jnp.array(mf.get_veff().exc))
dms.append(dmj)
ogds.append(dm.shape)
ao_evals.append(jnp.array(ao_eval))
gws.append(jnp.array(mf.grids.weights))
ts.append(jnp.array(mf.mol.intor('int1e_kin')))
vs.append(jnp.array(mf.mol.intor('int1e_nuc')))
mo_occs.append(jnp.array(mf.mo_occ))
hcs.append(jnp.array(mf.get_hcore()))
eris.append(jnp.array(mf.mol.intor('int2e')))
ss.append(jnp.linalg.inv(jnp.linalg.cholesky(mf.mol.intor('int1e_ovlp'))))
hologaps.append(jnp.array(mf.mo_energy[mf.mo_occ == 0][0] - mf.mo_energy[mf.mo_occ > 1][-1]))

In [7]:
class Band_gap_1shot_loss(eqx.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, model, ao_eval, gw, dm, eri, mo_occ, hc, s, ogd, refgap, mf, alpha0=0.7):
        vgf = lambda x: model(x, ao_eval, gw, mf)
        dmp, moep, mocp = xce.utils.get_dm_moe(dm, eri, vgf, mo_occ, hc, s, ogd, alpha0)
        
        efermi = moep[mf.mol.nelectron//2-1]
        moep -= efermi
        # print(moep)
        moep_gap = jnp.min(moep)
        # print(moep_gap)
        loss = jnp.sqrt( (moep_gap - refgap)**2)
        # print(loss)
        return jnp.sqrt( (moep_gap - refgap)**2)


In [8]:
#update docs, only input =2 ??? for MGGA? holdover from sebastian for some reason
xnet = xce.net.eX(n_input = 2, use = [1, 2], ueg_limit=True, lob=1.174)
# I guess use default LOB
cnet = xce.net.eC(n_input = 4, use = [2, 3], ueg_limit=True)
blankxc = xce.xc.eXC(grid_models = [xnet, cnet], level=3)
p = '/home/awills/Documents/Research/xcquinox/models/pretrained/scan'
# xc = eqx.tree_deserialise_leaves(os.path.join(p, 'xc.eqx'), blankxc)
xc = blankxc
nlxnet = xce.net.eX(n_input = 15, use = [], ueg_limit=True, lob=1.174)
nlcnet = xce.net.eC(n_input = 13, use = [], ueg_limit=True)

nlxc = xce.xc.eXC(grid_models = [nlxnet, nlcnet], level=4)

In [9]:
xct = xce.train.xcTrainer(model=nlxc, optim=optax.adamw(1e-2), steps=100, loss = Band_gap_1shot_loss(), do_jit=False)
newm = xct(1, xct.model, ao_evals, gws, dms, eris, mo_occs, hcs, ss, ogds, [-1.17], mfs)

Epoch 0
Epoch 0 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
Batch Loss = 0.44248081968472164
step=0, epoch_train_loss=0.44248081968472164
Epoch 1
Epoch 1 :: Batch 0/1


spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()


Batch Loss = 0.6345453200184403
step=1, epoch_train_loss=0.6345453200184403
Epoch 2
Epoch 2 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()


Batch Loss = 0.4832511573479339
step=2, epoch_train_loss=0.4832511573479339
Epoch 3
Epoch 3 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()


Batch Loss = 0.42676458909772474
step=3, epoch_train_loss=0.42676458909772474
Epoch 4
Epoch 4 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()


...

Epoch 95 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
Batch Loss = 0.3982178711837696
step=95, epoch_train_loss=0.3982178711837696


Epoch 96
Epoch 96 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
Batch Loss = 0.41750413006409315
step=96, epoch_train_loss=0.41750413006409315


Epoch 97
Epoch 97 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
Batch Loss = 0.4235645793009659
step=97, epoch_train_loss=0.4235645793009659
Epoch 98


Epoch 98 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
Batch Loss = 0.41200606814120877
step=98, epoch_train_loss=0.41200606814120877
Epoch 99


Epoch 99 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
Batch Loss = 0.4068303319267468
step=99, epoch_train_loss=0.4068303319267468


In [11]:
e1 = nlxc(dms[0], ao_evals[0], gws[0], mfs[0])
e2 = newm(dms[0], ao_evals[0], gws[0], mfs[0])

spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)


In [12]:
e1, e2

(Array(-2.57218399, dtype=float64), Array(-2.80120857, dtype=float64))

In [13]:
vgf1 = lambda x: xc(x, ao_evals[0], gws[0], mfs[0])
vgf2 = lambda x: newm(x, ao_evals[0], gws[0], mfs[0])
dm1, moe1, moc1 = xce.utils.get_dm_moe(dms[0], eris[0], vgf1, mo_occs[0], hcs[0], ss[0], ogds[0])
dm2, moe2, moc2 = xce.utils.get_dm_moe(dms[0], eris[0], vgf2, mo_occs[0], hcs[0], ss[0], ogds[0])

spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()


In [14]:
print(moe1 - moe1[mf.mol.nelectron//2-1])
print(moe2 - moe2[mf.mol.nelectron//2-1])

[-6.57726192e-01 -7.60975504e-02 -3.70073735e-07  0.00000000e+00
  1.02667394e-01  1.02667614e-01  3.18068487e-01  5.38626343e-01]
[-0.7616238  -0.07574408 -0.06304773  0.          0.08494971  0.10176512
  0.32993541  0.44945255]


In [ ]:
mf.mo_energy

In [ ]:
# vbmax = -99
for en in b1[0]:
    vb_k = en[cell.nelectron//2-1]
    print('This vb_k', vb_k)
    if vb_k > vbmax:
        vbmax = vb_k
e_kn = [en - vbmax for en in b1[0]]

In [ ]:
e_kn

In [ ]:
mf2 = scfp.RHF(cell)
e2 = mf2.kernel()

In [ ]:
cell.nelectron//2-1

In [ ]:
t1 = mf2.mo_energy 
t2 = mf2.mo_energy - mf2.mo_energy[cell.nelectron//2-1]

In [ ]:
t2[jnp.where(abs(t2[jnp.where( (t2 < 0) )[0]]) > 1e-4)[0]]

In [ ]:
dm2 = mf2.make_rdm1()

In [ ]:
dmk = mf.make_rdm1()

In [ ]:
mpr = MPRester(api_key = '')
mpid = 'mp-149'
# ret = mpr.get_bandstructure_by_material_id(mpid)
# rets = ret.structure.as_dict()
ret = mpr.get_structure_by_material_id(mpid, conventional_unit_cell=False)
rets = ret.as_dict()
at_coor_xyz = [ (i['species'][0]['element'], [-j for j in i['xyz']]) for i in rets['sites']]
at_coor_abc = [ (i['species'][0]['element'], [rets['lattice']['a']*j for j in i['abc']]) for i in rets['sites']]
cella = -np.asarray(rets['lattice']['matrix'])

In [ ]:
at_coor_xyz, cella

In [ ]:
cell = gtop.Cell()
cell.atom = at_coor_xyz
cell.a = cella.T
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build()
kpts = cell.make_kpts([2,2,2])
mf = scfp.KRHF(cell, kpts=kpts)
e = mf.kernel()

In [ ]:
cella